# Generate Data Graph from DB


### How to load DB

In [1]:
from pymongo import MongoClient


host = "172.25.0.98"
username = "olp"
password = "olppassword"

client = MongoClient(
    host=host,
    port=27017,
    username=username,
    password=password,
)

db_name = "yuri-pilot-cm"
collection_name = "ik_result"
db = client[db_name]
ik_result = db[collection_name]
docs = ik_result.find({})

### Check contents of DB

In [2]:
import pprint

next_doc = docs.next()
pprint.pprint(next_doc)

# or
# display(my_doc)

{'_id': ObjectId('64bfb1af278036647dafb1cc'),
 'carriage_joint_value': 0.0,
 'ee_offset': 0.01,
 'end_time': {'nsecs': 0, 'secs': 0},
 'frame_id': 'world',
 'inside_workspace': True,
 'joint_states': {'effort': [],
                  'header': {'frame_id': '',
                             'seq': 0,
                             'stamp': {'nsecs': 0, 'secs': 0}},
                  'name': ['arm0joint_a1',
                           'arm0joint_a2',
                           'arm0joint_a3',
                           'arm0joint_a4',
                           'arm0joint_a5',
                           'arm0joint_a6'],
                  'position': [1.9191180312730243,
                               0.8674844716027295,
                               2.455951166358247,
                               0.9543217747323756,
                               1.0967995913383455,
                               2.633666647613814],
                  'velocity': []},
 'name': 'SO0196',
 'planning_flag': 0

### Generate a fully connected graph
This will generate a fully connected graph.
For a set [goal, tool, robot, station, factory],

I tried make a graph with 

goal-tool,
tool-robot,
robot-station,
station-factory

But embedding result was not good.
So, I make a graph with all possible combinations from the set.

Refer, 
https://www.notion.so/Poincare-embedding-official-implementation-fb54f991e26047eab287b388c31ab095?pvs=4

However, this class will not be exported.
You will find an improved version of this class in OLP-SA-CLUSTERING -> hyperbolic_clustering.util.generate_data


In [3]:
from util.create_data import OlpDataGenerator

host = "172.25.0.98"
username = "olp"
password = "olppassword"

client = MongoClient(
    host=host,
    port=27017,
    username=username,
    password=password,
)

db_name = "yuri-pilot-cm"
collection_name = "ik_result"
odg = OlpDataGenerator(client=client, db_name=db_name, collection_name=collection_name)

In [5]:
odg.df

,id1,id2,weight
0,SO0196,tool11.R0.station_203r,1
1,SO0196,R0.station_203r,1
2,SO0196,station_203r,1
3,SO0196,factory,1
4,tool11.R0.station_203r,R0.station_203r,1
...,...,...,...
2876,SO0387,station_206r,1
2877,SO0387,tool22.R1.station_206r,1
2878,SO0387,tool23.R1.station_206r,1
2879,SO0387,tool24.R1.station_206r,1


### Export a graph

In [ ]:
export_filename = './data/olp_goals_connected_testfile.csv'
odg.export_graph(export_filename)

### Find how many tools are available for a goal

In [6]:
goal_count_df = odg.count_id('goal')
display(goal_count_df)

,id,count
0,SO0196,12
1,SO0197,4
2,SO0198,10
3,SO0199,2
4,SO0200,2
...,...,...
240,SO0264,6
241,SO0275,1
242,SO0276,2
243,SO0327,3


### Find how many times is a tool used

In [7]:
tool_count_df = odg.count_id('tool')
display(tool_count_df)

,id,count
0,tool11.R0.station_203r,24
1,tool12.R0.station_203r,15
2,tool13.R0.station_203r,25
3,tool14.R0.station_203r,34
4,tool15.R0.station_203r,31
5,tool16.R0.station_203r,33
6,tool18.R0.station_203r,37
7,tool21.R1.station_203r,119
8,tool22.R1.station_203r,59
9,tool23.R1.station_203r,28


### Which tools are used for goals

In [8]:
gt_list = odg.get_tool_list_for_goals()
display(gt_list)

,id,count,tool_list
0,SO0196,12,"[tool11.R0.station_203r, tool12.R0.station_203..."
1,SO0197,4,"[tool14.R0.station_203r, tool16.R0.station_203..."
2,SO0198,10,"[tool12.R0.station_203r, tool16.R0.station_203..."
3,SO0199,2,"[tool16.R0.station_203r, tool31.R2.station_206r]"
4,SO0200,2,"[tool23.R1.station_203r, tool24.R1.station_203r]"
...,...,...,...
240,SO0264,6,"[tool12.R0.station_206r, tool13.R0.station_206..."
241,SO0275,1,[tool22.R1.station_206r]
242,SO0276,2,"[tool22.R1.station_206r, tool23.R1.station_206r]"
243,SO0327,3,"[tool21.R1.station_206r, tool22.R1.station_206..."


### Necessary tools

In [9]:
count_exp = "(count == 1)"
q_ind = gt_list.query(count_exp).index.values

temp_necessary_tools = gt_list.iloc[q_ind, :]['tool_list']

temp_necessary_tools.iloc[0][0]
necessary_tools = []
for i in range(len(temp_necessary_tools)):
    count = 0
    for j in range(len(necessary_tools)):
        if necessary_tools[j] == temp_necessary_tools.iloc[i][0]:
            count += 1
    if count == 0:
        necessary_tools.append(temp_necessary_tools.iloc[i][0])

necessary_tools.sort()
necessary_tools

['tool22.R1.station_206r',
 'tool23.R1.station_206r',
 'tool25.R1.station_206r',
 'tool32.R2.station_206r',
 'tool33.R2.station_206r']

In [10]:
newig = odg.get_hierarchy_names()
print(newig)

                id
0          factory
1  R0.station_203r
2     station_203r
3  R1.station_203r
4  R0.station_206r
5     station_206r
6  R1.station_206r
7  R2.station_206r
